# Optimized Model

In [1]:
# Dependencies
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from keras import backend as K


In [2]:
# Data
app_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
app_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop unneccessary columns
app_df.drop(columns=['EIN', 'NAME'], inplace=True)
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Counting unique values in classification column
class_count = app_df['CLASSIFICATION'].value_counts()
class_count

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [5]:
# Binning/Bucketing categories in classification column
cutoff = 1000
class_replacing = class_count[class_count <= cutoff].index.tolist()
for classes in class_replacing:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(classes, "Other")

app_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [6]:
# Counting vallues in application type column
app_count = app_df['APPLICATION_TYPE'].value_counts()
app_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Binning/bucketing categorical values in application type column
cutoff = 500
app_replace = app_count[app_count <= cutoff].index.tolist()

for app in app_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

app_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Encoding remaining categorical values into numerical values
encoded_df = pd.get_dummies(app_df, dtype=int)
encoded_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# Split data into X and y as well as train/test 
X = encoded_df.drop(columns='IS_SUCCESSFUL')
y = encoded_df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [10]:
# Scaling data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# First Optimization

In [11]:
# Compile neural network
input_features = len(X_train.columns)
layer_one = 110
layer_two = 60
layer_three = 60
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(units=layer_one, activation="relu", input_dim=input_features))
model.add(tf.keras.layers.Dense(units=layer_two, activation="relu"))
model.add(tf.keras.layers.Dense(units=layer_three, activation="relu"))
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
model.summary()

C:\Users\New User\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 110)                 │           4,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 60)                  │           6,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 60)                  │           3,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,221 (59.46 KB)

 Trainable params: 15,221 (59.46 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train/fit model
model_fit = model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7058 - loss: 0.5878
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7313 - loss: 0.5552
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7303 - loss: 0.5527
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7312 - loss: 0.5496
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7325 - loss: 0.5454
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7292 - loss: 0.5502
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7296 - loss: 0.5462
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7294 - loss: 0.5490
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7362 - loss: 0.5420
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7320 - loss: 0.5424
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7317 - loss: 0.5441
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [14]:
# Evaluate
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Accuracy: {model_accuracy}, Loss: {model_loss}")

268/268 - 1s - 2ms/step - accuracy: 0.7298 - loss: 0.5790
Accuracy: 0.7297959327697754, Loss: 0.5789707899093628


In [15]:
# Clear keras session from memory prior to next optimization
K.clear_session()

# Second Optimization

In [16]:
# Compile neural network
input_features = len(X_train.columns)
layer_one = 125
layer_two = 45
layer_three = 45
layer_four = 30
model_two = tf.keras.models.Sequential()

model_two.add(tf.keras.layers.Dense(units=layer_one, activation="tanh", input_dim=input_features))
model_two.add(tf.keras.layers.Dense(units=layer_two, activation="tanh"))
model_two.add(tf.keras.layers.Dense(units=layer_three, activation="tanh"))
model_two.add(tf.keras.layers.Dense(units=layer_four, activation="tanh"))
model_two.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
model_two.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 125)                 │           5,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 45)                  │           5,670 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 45)                  │           2,070 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 30)                  │           1,380 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,651 (57.23 KB)

 Trainable params: 14,651 (57.23 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model_two.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train/fit model
model_two_fit = model_two.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7148 - loss: 0.5808
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7236 - loss: 0.5623
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7307 - loss: 0.5536
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7264 - loss: 0.5562
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7302 - loss: 0.5536
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7299 - loss: 0.5471
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7364 - loss: 0.5436
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7302 - loss: 0.5448
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7330 - loss: 0.5455
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7349 - loss: 0.5448
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7322 - loss: 0.5445
Epoch 12/150
804/804 ━━━━━━━━━━━━━━━━━━━━

In [19]:
# Evaluate
model_loss, model_accuracy = model_two.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Accuracy: {model_accuracy}, Loss: {model_loss}")

268/268 - 1s - 2ms/step - accuracy: 0.7286 - loss: 0.5636
Accuracy: 0.7286297082901001, Loss: 0.5635848045349121


In [20]:
# Clear keras session from memory prior to next optimization
K.clear_session()

# Third Optimization

In [21]:
# Compile neural network
input_features = len(X_train.columns)
layer_one = 120
layer_two = 50
layer_three = 50
layer_four = 50
model_three = tf.keras.models.Sequential()

model_three.add(tf.keras.layers.Dense(units=layer_one, activation="relu", input_dim=input_features))
model_three.add(tf.keras.layers.Dense(units=layer_two, activation="relu"))
model_three.add(tf.keras.layers.Dense(units=layer_three, activation="relu"))
model_three.add(tf.keras.layers.Dense(units=layer_four, activation="relu"))
model_three.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
model_three.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 120)                 │           5,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           6,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,481 (64.38 KB)

 Trainable params: 16,481 (64.38 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model_three.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train/fit model
model_three_fit = model_three.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7068 - loss: 0.5858
Epoch 2/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7325 - loss: 0.5516
Epoch 3/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7324 - loss: 0.5484
Epoch 4/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7293 - loss: 0.5538
Epoch 5/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7303 - loss: 0.5476
Epoch 6/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7357 - loss: 0.5438
Epoch 7/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7355 - loss: 0.5442
Epoch 8/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7388 - loss: 0.5397
Epoch 9/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7373 - loss: 0.5411
Epoch 10/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7357 - loss: 0.5438
Epoch 11/125
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7377 - loss: 0.5403
Epoch 12/125
804/804 ━━━━━━━━━━━━━━━━━━━━

In [24]:
# Evaluate
model_loss, model_accuracy = model_three.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Accuracy: {model_accuracy}, Loss: {model_loss}")

268/268 - 1s - 2ms/step - accuracy: 0.7282 - loss: 0.5922
Accuracy: 0.7281632423400879, Loss: 0.5922452807426453


In [25]:
# Save Optimized Model
filepath = "AlphabetSoupCharity_Optimization.keras"
model.save(filepath)